In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("dbfs:/FileStore/shared_uploads/mahdas@ebay.com/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.7, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8983515819505853

In [0]:
dbfs:/FileStore/shared_uploads/mahdas@ebay.com/movies.dat
dbfs:/FileStore/shared_uploads/mahdas@ebay.com/itemusermat

In [0]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
import pandas as pd
data = sc.textFile("dbfs:/FileStore/shared_uploads/mahdas@ebay.com/itemusermat")
businessDF = spark.read \
    .option("sep", "::") \
    .csv("dbfs:/FileStore/shared_uploads/mahdas@ebay.com/movies.dat") \
    .withColumnRenamed("_c0", "movie_id_") \
    .withColumnRenamed("_c1", "title") \
    .withColumnRenamed("_c2", "genre") \
    .dropDuplicates()
parsedData = data.map(lambda line: array([int(x) for x in line.split(' ')[1:]]))
clusters = KMeans.train(parsedData, 10, maxIterations=10, initializationMode="random")

def getcluster(point):
    center = clusters.predict(point[1:])
    return (point[0], center)

WSSSE = data.map(lambda line: array([int(x) for x in line.split(' ')])).map(lambda point: getcluster(point)).collect()
deptColumns = ["movie_id","cluster"]
te = pd.DataFrame(list(WSSSE), columns = deptColumns)
deptDF = spark.createDataFrame(te)

df = deptDF.join(businessDF, businessDF.movie_id_ == deptDF.movie_id, how="inner")

window = Window.partitionBy(df['cluster']).orderBy(df['movie_id'].desc())
  
df.select('*', rank().over(window).alias('rank'))\
  .filter(col('rank') <= 5)\
  .show(100)

+--------+-------+---------+--------------------+--------------------+----+
movie_id|cluster|movie_id_| title| genre|rank|
+--------+-------+---------+--------------------+--------------------+----+
 2987| 0| 2987|Who Framed Roger ...|Adventure|Animati...| 1|
 2804| 0| 2804|Christmas Story, ...| Comedy|Drama| 2|
 2194| 0| 2194|Untouchables, The...| Action|Crime|Drama| 3|
 1968| 0| 1968|Breakfast Club, T...| Comedy|Drama| 4|
 1961| 0| 1961| Rain Man (1988)| Drama| 5|
 3809| 1| 3809|What About Bob? (...| Comedy| 1|
 3526| 1| 3526| Parenthood (1989)| Comedy|Drama| 2|
 3256| 1| 3256|Patriot Games (1992)| Action|Thriller| 3|
 3255| 1| 3255|League of Their O...| Comedy|Drama| 4|
 3253| 1| 3253|Wayne's World (1992)| Comedy| 5|
 3901| 2| 3901| Duets (2000)| Comedy|Drama| 1|
 3839| 2| 3839|Phantasm IV: Obli...| Horror| 2|
 3838| 2| 3838|Phantasm III: Lor...| Horror| 3|
 3835| 2| 3835| Crush, The (1993)| Thriller| 4|
 3834| 2| 3834| Bronco Billy (1980)|Adventure|Drama|R...| 5|
 2858| 3| 2858|American Beauty (...| Comedy|Drama| 1|
 2997| 4| 2997|Being John Malkov...| Comedy| 1|
 1288| 4| 1288|This Is Spinal Ta...|Comedy|Drama|Musical| 2|
 1235| 4| 1235|Harold and Maude ...| Comedy| 3|
 1230| 4| 1230| Annie Hall (1977)| Comedy|Romance| 4|
 3836| 5| 3836|Kelly's Heroes (1...| Action|Comedy|War| 1|
 3827| 5| 3827|Space Cowboys (2000)| Action|Sci-Fi| 2|
 3826| 5| 3826| Hollow Man (2000)|Horror|Sci-Fi|Thr...| 3|
 3802| 5| 3802| Freejack (1992)| Action|Sci-Fi| 4|
 3785| 5| 3785| Scary Movie (2000)| Comedy|Horror| 5|
 3811| 6| 3811|Breaker Morant (1...| Drama|War| 1|
 1233| 6| 1233|Boat, The (Das Bo...| Action|Drama|War| 2|
 3819| 7| 3819| Tampopo (1986)| Comedy| 1|
 3814| 7| 3814|Love and Death (1...| Comedy| 2|
 3788| 7| 3788| Blowup (1966)| Drama|Mystery| 3|
 3742| 7| 3742|Battleship Potemk...| Drama|War| 4|
 3741| 7| 3741| Badlands (1973)| Crime|Drama| 5|
 3699| 8| 3699| Starman (1984)|Adventure|Drama|R...| 1|
 3527| 8| 3527| Predator (1987)|Action|Sci-Fi|Thr...| 2|
 2985| 8| 2985| Robocop (1987)| Action|Crime|Sci-Fi| 3|
 2872| 8| 2872| Excalibur (1981)|Action|Drama|Fant...| 4|
 2455| 8| 2455| Fly, The (1986)| Horror|Sci-Fi| 5|
 3755| 9| 3755|Perfect Storm, Th...|Action|Adventure|...| 1|
 3753| 9| 3753| Patriot, The (2000)| Action|Drama|War| 2|
 3751| 9| 3751| Chicken Run (2000)|Animation|Childre...| 3|
 1617| 9| 1617|L.A. Confidential...|Crime|Film-Noir|M...| 4|
+--------+-------+---------+--------------------+--------------------+----+

In [0]:
from pyspark.ml.recommendation import ALS

# Let's initialize our ALS learner
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12]
errors = [0, 0, 0]
models = [0, 0, 0]
err = 0
min_error = float('inf')
best_rank = -1
for rank in ranks:
  # Create the model with these parameters.
  model = als.fit(training)
  # Run the model to create a prediction. Predict against the validation_df.
  predict_df = model.transform(test)

  # Remove NaN values from prediction (due to SPARK-14489)
  predicted_ratings_df = predict_df.filter(predict_df.prediction != float('nan'))

  # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
  error = reg_eval.evaluate(predicted_ratings_df)
  errors[err] = error
  models[err] = model
  print ('For rank %s the RMSE is %s' % (rank, error))
  if error < min_error:
    min_error = error
    best_rank = err
  err += 1

als.setRank(ranks[best_rank])
print('The best model was trained with rank %s' % ranks[best_rank])
my_model = models[best_rank]

For rank 4 the RMSE is 0.8983515819505853
For rank 8 the RMSE is 0.8983515819505853
For rank 12 the RMSE is 0.8983515819505853
The best model was trained with rank 4